# crystal viewer

In [17]:
%matplotlib widget
  
# importing required libraries
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

#data tools
import pandas as pd
from pymatgen.ext.matproj import MPRester
#from pymatgen.vis import structure_chemview as viz

#simtool loading and interface
from simtool import findInstalledSimToolNotebooks,searchForSimTool
from simtool import getSimToolInputs,getSimToolOutputs,Run

#user interface utilities 
import os, stat
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
import numpy as np

In [3]:
with open(os.path.expanduser("~/.mpkey.txt"), "r+") as file:
    apikey = file.readline()
rester = MPRester(apikey)
sc_dicts = rester.query({ "crystal_system": "cubic"},
                        ["task_id","pretty_formula","formula","elements","e_above_hull", "spacegroup.number", "band_gap", "crystal_system"])
sc_df = pd.DataFrame(sc_dicts)

In [143]:
# import POSCAR file

mpid = "mp-2133"
struct = rester.get_structure_by_material_id(mpid, final = False, conventional_unit_cell=True)
POSCAR_str = struct.to(fmt = "poscar")

print(mpid)

lines = POSCAR_str.split('\n')
print(POSCAR_str)

# get the lattice information

lattice = lines[1]
cell_vectors = np.array([lines[2].split() , lines[3].split() , lines[4].split()]).astype(float)

# get the list of sites

sites = []
for line in lines[8:]:
    if not line:
        break
    sites.append([line.split()[0],line.split()[1],line.split()[2]])

# convert from fractional to xyz

sites = np.array(sites).astype(float)
xyz = np.matmul(sites,cell_vectors).transpose()

# get the coordinates of the box

corners = np.array([[0,1,1,0,0,0,0,1,1,0,0,1,1,1,1,0,0],[0,0,1,1,0,0,0,0,1,1,0,0,0,1,1,1,1],[0,0,0,0,0,1,1,1,1,1,1,1,0,0,1,1,0]]).T
cell = np.matmul(corners,cell_vectors).transpose()

# get a color dictionary

elements = lines[5].split()
atoms = lines[6].split()
hues = ['b','g','r','y']
colors = []
for i, atom in enumerate(atoms):
    colors.extend([elements[i]]*int(atoms[i]))

zip_iterator = zip(elements,hues)
color_dict = dict(zip_iterator)
print(color_dict)

# creating figure

plt.close('all')

fig = plt.figure()
ax = Axes3D(fig)
  
plot = ax.scatter(xyz[0],xyz[1],xyz[2], color=[color_dict[i] for i in colors], s = 256)
plot = ax.plot(cell[0],cell[1],cell[2],color='black')
  
ax.set_title("POSCAR")
ax.axis('off')
  
# displaying the plot
plt.show()

mp-2133
Zn2 O2
1.0
1.592050 -2.757511 0.000000
1.592050 2.757511 0.000000
0.000000 0.000000 5.155100
Zn O
2 2
direct
0.666667 0.333333 0.500000 Zn
0.333333 0.666667 0.000000 Zn
0.666667 0.333333 0.880310 O
0.333333 0.666667 0.380310 O

{'Zn': 'b', 'O': 'g'}


FigureCanvasNbAgg()